In [1]:
### IMPORTS

#import libs, and the Node and MCTS modules
# import sys
# sys.path.insert(1, '../Desktop/Hex_bot_group/agents/MCTSAgent/')

from mctsNode import mctsNode
import mctsSearchFull_NN as mcts

In [4]:
### IRRELEVANT to NN side

# check that _winner() works
winningBoard = [
    [1,0,0],
     [1,0,0],
      [1,1,1]
]
mcts._winner(winningBoard)


# Need to write an interpreter to work with this kind of board input outputs
#
#     R B 0 0
#      R R 0 0
#       B R R 0
#        X X 0 0
#

1

In [53]:
### HELPER PRINTING FUNC

# Nicer printing out function
# Not amazing

totallen = 10
halflen = 5
def niceprint(returnboard):
    for i in range(len(returnboard)):
        line = ""
        line+= halflen*i*" "
        for j in range(len(returnboard)):
            el = str(returnboard[i][j])
            toappend = totallen - len(el)
            line += el
            line += toappend*" "
        print(line)
        
totallen2 = 8
halflen2 = 4
def niceprint2(returnboard):
    for i in range(len(returnboard)):
        line = ""
        line+= halflen*i*" "
        for j in range(len(returnboard)):
            el = str(returnboard[i][j])
            toappend = totallen - len(el)
            line += el
            line += toappend*" "
        print(line)
        print("")

In [78]:
### THE GOOD STUFF

board = [
    [0,0,0,0],
     [0,0,0,0],
     [0,-1,1,-1],
       [0,0,1,-1],
]


bigboard = [
    [1,0,0,-1,0],
     [0,0,0,0,0],
      [0,0,0,-1,0],
       [0,0,1,0,0],
        [1,0,0,1,-1]
]

verybigboard = [
    [ 1, 0, 0,-1, 0, 0, 0],
     [ 0, 0, 0, 0, 0, 0, 0],
      [ 0, 0, 0,-1, 0, 0, 0],
       [ 0, 0, 1, 0, 0, 0, 0],
        [ 1, 0, 0, 1,-1, 0, 0],
         [ 0, 0, 0,-1, 0, 0, 0],
          [ 0, 0, 1, 0, 0, 0, 0]
]


root = mctsNode(None, bigboard, None)

In [77]:

# call the search with root containing the board, #iterations, #playouts per new node, C

# niceprint(mcts.mctsSearch(root,9,10,0.9))
# # note how here nothing changes after 9 iterations (full depth of search tree?)

root = mctsNode(None, bigboard, None)
niceprint(mcts.mctsSearch(root,1000,1,0.4))

# so the tuples in the returned board of mctsSearch are Q, N
# Where practically we kind of mainly care about N.. because N is affected by Q within the algorithm



1         (10, 19)  (20, 29)  -1        (747, 796)
     (28, 39)  (79, 96)  (42, 54)  (28, 39)  (212, 239)
          (16, 24)  (183, 208)(12, 19)  -1        (18, 29)  
               (114, 134)(12, 19)  1         (3, 9)    (6, 14)   
                    1         (4, 9)    (11, 19)  1         -1        


In [83]:
### A BETTER REPRESENTATION (?): 
def improveboard(rboard):
    for i in range(len(rboard)):
        for j in range(len(rboard)):
            el = rboard[i][j]
            if type(el) is tuple:
                rboard[i][j] = el[0]+el[1]
    return rboard
                
 
root = mctsNode(None, verybigboard, None)     

#                                 root,  iterations,  playouts per node,  c (0-1 usually)
niceprint2(improveboard(mcts.mctsSearch(root,100,10,0.1)))


1         14        318       -1        307       14        714       

     15        1491      15        319       15        14        310       

          15        7505      323       -1        15        14        15        

               15        317       1         516       15        13        15        

                    1         321       321       1         -1        14        14        

                         714       315       11        -1        14        14        15        

                              1888      323       1         15        14        14        510       

